In [6]:
import pandas as pd
import plotly.graph_objs as go


In [7]:
def load_and_prepare_data(file_path):
    """
    Load SPY data from a CSV file, parse dates, and calculate EMA(21) and MA(50).
    """
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df.sort_values('time', inplace=True)

    # Calculate EMA(21) and MA(50)
    df['EMA_21'] = df['close'].ewm(span=21, adjust=False).mean()
    df['MA_50'] = df['close'].rolling(window=50).mean()
    return df


In [8]:
def implement_trading_strategy(df):
    """
    Detect buy/sell signals based on EMA(21) and MA(50) crossovers.
    Returns a DataFrame with trade entries and exits.
    """
    trades = []  # List to store each trade
    position = None  # Tracks if a position is open ("buy") or not (None)

    for i in range(1, len(df)):
        # Buy signal: MA_50 crosses above EMA_21
        if df['MA_50'].iloc[i] > df['EMA_21'].iloc[i] and df['MA_50'].iloc[i - 1] <= df['EMA_21'].iloc[i - 1]:
            if position is None:  # Only buy if not already holding
                position = "buy"
                entry_time = df['time'].iloc[i]
                entry_price = df['close'].iloc[i]

        # Sell signal: EMA_21 crosses above MA_50
        elif df['EMA_21'].iloc[i] > df['MA_50'].iloc[i] and df['EMA_21'].iloc[i - 1] <= df['MA_50'].iloc[i - 1]:
            if position == "buy":  # Only sell if there's an open position
                position = None
                exit_time = df['time'].iloc[i]
                exit_price = df['close'].iloc[i]
                pnl = (exit_price / entry_price) - 1
                trades.append({
                    "enter_time": entry_time,
                    "exit_time": exit_time,
                    "enter_price": entry_price,
                    "exit_price": exit_price,
                    "pnl": pnl
                })

    # Close any open position at the last date
    if position == "buy":
        exit_time = df['time'].iloc[-1]
        exit_price = df['close'].iloc[-1]
        pnl = (exit_price / entry_price) - 1
        trades.append({
            "enter_time": entry_time,
            "exit_time": exit_time,
            "enter_price": entry_price,
            "exit_price": exit_price,
            "pnl": pnl
        })

    trades_df = pd.DataFrame(trades)
    return trades_df


In [9]:
# File path to the CSV data
file_path = '/content/drive/MyDrive/BATS_SPY_1W_homework.csv'

# Load and prepare data
df = load_and_prepare_data(file_path)

# Apply the trading strategy and get trades
trades_df = implement_trading_strategy(df)

# Display the trades
print(trades_df)


                      enter_time                     exit_time  enter_price  \
0  1970-01-01 00:00:00.767280600 1970-01-01 00:00:00.781795800     45.09375   
1  1970-01-01 00:00:00.787242600 1970-01-01 00:00:00.787847400     45.75000   
2  1970-01-01 00:00:00.788452200 1970-01-01 00:00:00.789057000     45.56250   
3  1970-01-01 00:00:00.972307800 1970-01-01 00:00:01.053351000    139.28125   
4  1970-01-01 00:00:01.098106200 1970-01-01 00:00:01.099319400    109.99001   
5  1970-01-01 00:00:01.199716200 1970-01-01 00:00:01.249306200    140.14999   
6  1970-01-01 00:00:01.281360600 1970-01-01 00:00:01.286199000    108.31000   
7  1970-01-01 00:00:01.314019800 1970-01-01 00:00:01.328538600    117.97000   
8  1970-01-01 00:00:01.441719000 1970-01-01 00:00:01.462800600    196.74000   
9  1970-01-01 00:00:01.544452200 1970-01-01 00:00:01.554730200    260.47000   
10 1970-01-01 00:00:01.584970200 1970-01-01 00:00:01.594647000    253.42000   
11 1970-01-01 00:00:01.650893400 1970-01-01 00:00:01